In [1]:
#训练体育类评论模型

In [2]:
import pandas as pd
import os
from SwordTextPreprocessing import *

In [3]:
data = pd.DataFrame(columns = ['评论内容'])

#循环切片、合并
for i in range(0, 4):
    part_data_filename = 'data_tiyu_' + str(i) + '.csv'
    part_data_filename = os.path.join(os.getcwd(), 'data', 'data_tiyu', part_data_filename)
    part_data = pd.read_csv(part_data_filename)
    part_data = pd.DataFrame(part_data['评论内容'], columns = ['评论内容'])
    #合并
    data = pd.concat([data, part_data], ignore_index = True)
    
data_filename = os.path.join(os.getcwd(), 'data', 'data_tiyu', 'tiyu_all.csv')
data.to_csv(data_filename, index = False)

In [4]:
#机械去重、去除 @ 、去除短句
#很诡异，单独跑时 jupyter 中把上一个 cell 里的 data dataFrame 变成 series了
data = data['评论内容'].apply(text_compress_save2)
#单独跑一段用此
#data = data.apply(text_compress)
compress_filename = os.path.join(os.getcwd(), 'data', 'data_tiyu', 'tiyu_compress.csv')
data.to_csv(compress_filename, index = False)

data = data.apply(remove_word_by_head)
removeAt_filename = os.path.join(os.getcwd(), 'data', 'data_tiyu', 'tiyu_removeAt.csv')
data.to_csv(removeAt_filename, index = False)data = data.apply(remove_word_by_head)

data = data.apply(short_text_none)
data = data.dropna(axis = 0, how = 'any')
cutshort_filename = os.path.join(os.getcwd(), 'data', 'data_tiyu', 'tiyu_cutshort.csv')
data.to_csv(cutshort_filename, index = False)

In [5]:
#手工对data_cutshort.csv加评论分类 2为积极，0为消极, 1为中立，-1为无关，得到tiyu_train.csv

In [6]:
#读取训练数据，分词
train_filename = os.path.join(os.getcwd(), 'data', 'data_tiyu', 'tiyu_train.csv')
train_data = pd.read_csv(train_filename)
train_data = train_data.dropna(axis = 0, how = 'any')
train_data['jieba'] = train_data['评论内容'].apply(cut_jieba)
print(train_data['jieba'].head())

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
Loading model cost 0.698 seconds.
Prefix dict has been built successfully.


0    对光 詹姆斯 有 黑点 ， 科比 没 黑点 ， 笑 死 了 ， 都 不好意思 说 ， 知道 ...
1       哇   你 妈 不会 是 鹰 郡 事件 的 女主角 吧 ？ 💰 拿到 了 不 ？ [ 色 ]
2                              看   就会 这句   有 烧焦 密 那味 了
3                              科比 打篮球 还 行   但开 飞机 不 太行
4    这 哥们 唠嗑 活着 时候 不说 ， 现在 不 在 了 敢 说 了 ？ 你 咋 不 说 科比...
Name: jieba, dtype: object


In [7]:
#去除停用词、生成CountVectorizer类（用于生成词向量）
stopwords_filename = os.path.join(os.getcwd(), 'stopwords', 'hit_stopwords.txt')
stopwords = get_stopwords(stopwords_filename)

from sklearn.feature_extraction.text import CountVectorizer
word_vector = CountVectorizer(token_pattern = u'(?u)\\b[^\\d\\W]\\w+\\b', stop_words = stopwords)

In [8]:
#利用CountVectorizer转成词向量
X_train = train_data['jieba']
X_train_vector = word_vector.fit_transform(X_train)
y_train = train_data['mood']

import pickle
# 保存经过fit的vectorizer，预测时使用
countvectorizer_filename = os.path.join(os.getcwd(), 'data', 'data_tiyu', 'countvectorizer_tiyu.pkl')
with open(countvectorizer_filename, 'wb') as fp:
    pickle.dump(word_vector.vocabulary_, fp)

d:\python\lib\site-packages\sklearn\feature_extraction\text.py:382: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [9]:
#多项式贝叶斯 用tiyu_train.csv的数据进行训练
from sklearn.naive_bayes import MultinomialNB
multiNB_model = MultinomialNB()
multiNB_model.fit(X_train_vector, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
import joblib
model_filename = os.path.join(os.getcwd(), 'data', 'data_tiyu', 'multiNB_tiyu.pkl')
#存入文件
joblib.dump(multiNB_model, model_filename)

['D:\\projects\\JupyterWorks\\Test\\data\\data_tiyu\\multiNB_tiyu.pkl']

In [1]:
#测试
import time
time_start = time.process_time()

import os
from SwordTextPreprocessing import *
data_folder = 'data_tiyu'
countvectorizer_filename = os.path.join(os.getcwd(), 'data', data_folder, 'countvectorizer_tiyu.pkl')
model_filename = os.path.join(os.getcwd(), 'data', data_folder, 'multiNB_tiyu.pkl')
test_filename = os.path.join(os.getcwd(), 'data', data_folder, 'test_meixi.csv')
predict_filename = os.path.join(os.getcwd(), 'data', data_folder, 'predict_meixi.csv')
column_name = '评论内容'
test_preprecessing_predict(test_filename, column_name, predict_filename, countvectorizer_filename, model_filename)
#打印整个预测过程时间
print(time.process_time() - time_start)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
Loading model cost 0.676 seconds.
Prefix dict has been built successfully.


1.75


In [12]:
#比较加载的词向量模型是否还需使用停用词

#比较两个dataframe是否一致 结论:一致
#from pandas.testing import assert_frame_equal
#print(assert_frame_equal(data1, data2))